# Metaprogramming

From Wikipedia:

> In computer programming, **homoiconicity** (from the Greek words homo- meaning "the same" and icon meaning "representation") is a property of some programming languages. A language is **homoiconic** if a program written in it can be manipulated as data using the language, and thus the program's internal representation can be inferred just by reading the program itself. For example, a Lisp program is written as a regular Lisp list, and can be manipulated by other Lisp code.[1] In homoiconic languages, all code can be accessed and transformed as data, using the same representation. This property is often summarized by saying that the language treats "code as data".

Julia is homoiconic. In Julia, program code can be represented by a Julia data structure called an expression.

In [1]:
Meta.parse("2 + 3")

:(2 + 3)

In [2]:
typeof(ans)

Expr

In [3]:
two_plus_three = :(2 + 3)

:(2 + 3)

This is known as *quoting* and `:` is the `quote` operator.

Code with more than one line can be quoted like this:

In [4]:
quote
    a = 42
    b = a^2
    a - b
end

quote
    #= In[4]:2 =#
    a = 42
    #= In[4]:3 =#
    b = a ^ 2
    #= In[4]:4 =#
    a - b
end

In [5]:
eval(two_plus_three)

5

We can use the `dump` function to see how any value in Julia is represented.

In [6]:
dump(two_plus_three)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 2
    3: Int64 3


`head` indicates that this expression is a function call. `args` is an array containing the function and its arguments.

In [7]:
two_plus_three.args[1]

:+

Let's make a copy of the expression and modify the first `arg`

In [8]:
two_minus_three = copy(two_plus_three)
two_minus_three.args[1] = :-

:-

In [9]:
two_plus_three

:(2 + 3)

Now we can look at and evaluate the modified expression.

In [10]:
two_minus_three

:(2 - 3)

In [11]:
eval(two_minus_three)

-1

## Macros

Using the above and just a little more metaprogramming machinery, we can create powerful *macros*.

Macros run when the code is parsed, and generate expressions that are compiled directly, rather than requiring a call to `eval` at runtime.

Here is an example.

In [12]:
macro timeit(ex)
    quote
        local t0 = time()
        local val = $(esc(ex))
        local t1 = time()
        println("elapsed time: ", t1-t0, " seconds")
        val
    end
end

@timeit (macro with 1 method)

In [13]:
@timeit factorial(20)

elapsed time: 0.018849849700927734 seconds


2432902008176640000

Note: Julia comes with a built-in `@time` macro.

In [14]:
@time factorial(20)

  0.000001 seconds


2432902008176640000

Note that there is a `@macroexpand` macro (and other facilities) that can be extremely useful for debugging macros.

In [15]:
@macroexpand @timeit factorial(20)

quote
    #= In[12]:3 =#
    local var"#37#t0" = Main.time()
    #= In[12]:4 =#
    local var"#38#val" = factorial(20)
    #= In[12]:5 =#
    local var"#39#t1" = Main.time()
    #= In[12]:6 =#
    Main.println("elapsed time: ", var"#39#t1" - var"#37#t0", " seconds")
    #= In[12]:7 =#
    var"#38#val"
end

# Taylor series example

Based on an example by Mike J Innes.

## Original sin

Here's a more practical example. Consider the following definition of the `sin` function, based on the [Taylor series](https://en.wikipedia.org/wiki/Taylor_series).

$$sin(x) = \sum_{k=0}^{\infty} \frac{(-1)^k}{(1+2k)!} x^{1+2k}$$

or

$$sin(x) = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} ...$$

*Aside:* The following code uses a generator expression, which is a comprehension written without the square brackets. Generator expressions produce values on demand without storing them.

In [16]:
mysin(x) = sum((-1)^k/factorial(1+2k) * x^(1+2k) for k = 0:9)

mysin (generic function with 1 method)

In [17]:
mysin(0.5), sin(0.5)

(0.479425538604203, 0.479425538604203)

To see where we are right now, we'll benchmark it.

In [18]:
using BenchmarkTools
@benchmark mysin(0.5)

BechmarkTools.Trial: 10000 samples with 12 evaluations.
 Range (min … max):  991.583 ns …  3.695 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):       1.062 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):     1.050 μs ± 73.347 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▁     ▅▆       █▇       ▅▇▁        ▂         ▂▂          ▂ ▂
  ███▁▁▁▃▃███▃▃▃▃▁▇██▇▁▃▁▄▃▃███▁▁▁▁▃▁▁██▅▁▁▁▁▅▁▅▃██▅▅▆▆▆▃▄▄▃▆█ █
  992 ns        Histogram: log(frequency) by time      1.24 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Right now, this is much slower than it could be. The reason is that we're looping over `k`, which is relatively expensive. It'd be much faster to write out:

In [19]:
mysin(x) = x - x^3/6 + x^5/120 + x^7/5040 # + ...

mysin (generic function with 1 method)

But this is tedious to write, and no longer looks like the original Taylor series. It's harder to tell if we've made a mistake, and we easily modify it. Is there a way to get the best of both worlds?

How about getting Julia to write out that code for us?

To start with, let's consider a symbolic version of the `+` function.

In [20]:
plus(a, b) = :($a + $b)

plus (generic function with 1 method)

This is a function that returns an expression.

In [21]:
plus(1, 2)

:(1 + 2)

With `plus` we can do more interesting things, like symbolic `sum`:

In [22]:
reduce(+, 1:10)

55

In [23]:
reduce(plus, 1:10)

:(((((((((1 + 2) + 3) + 4) + 5) + 6) + 7) + 8) + 9) + 10)

In [24]:
eval(ans)

55

Given that, we can also sum over symbolic variables.

In [25]:
reduce(plus, [:(x^2), :x, 1])

:((x ^ 2 + x) + 1)

This gives us an important piece of the puzzle, but we also need to figure out _what_ we're summing. Let's crate a symbolic version of the Taylor series above, which interpolates the value of `k`.

In [26]:
k = 3
:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k)))

:((-1 * x ^ 7) / 5040)

Now we have one term, we can generate as many as we like.

In [27]:
terms = [:($((-1)^k) * x^$(1+2k) / $(factorial(1+2k))) for k = 0:9]

10-element Vector{Expr}:
 :((1 * x ^ 1) / 1)
 :((-1 * x ^ 3) / 6)
 :((1 * x ^ 5) / 120)
 :((-1 * x ^ 7) / 5040)
 :((1 * x ^ 9) / 362880)
 :((-1 * x ^ 11) / 39916800)
 :((1 * x ^ 13) / 6227020800)
 :((-1 * x ^ 15) / 1307674368000)
 :((1 * x ^ 17) / 355687428096000)
 :((-1 * x ^ 19) / 121645100408832000)

And sum them –

In [28]:
taylor_series = reduce(plus, terms)

:((((((((((1 * x ^ 1) / 1 + (-1 * x ^ 3) / 6) + (1 * x ^ 5) / 120) + (-1 * x ^ 7) / 5040) + (1 * x ^ 9) / 362880) + (-1 * x ^ 11) / 39916800) + (1 * x ^ 13) / 6227020800) + (-1 * x ^ 15) / 1307674368000) + (1 * x ^ 17) / 355687428096000) + (-1 * x ^ 19) / 121645100408832000)

And create a function definition out of it:

In [29]:
eval(:(mysin(x) = $taylor_series))

mysin (generic function with 1 method)

In [30]:
mysin(0.5), sin(0.5)

(0.479425538604203, 0.479425538604203)

In [31]:
@benchmark mysin(0.5)

BechmarkTools.Trial: 10000 samples with 115 evaluations.
 Range (min … max):  715.391 ns …  1.334 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     761.739 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   764.644 ns ± 47.350 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▃█   ▂█   ▁▃▁  ▃▃▅     ▃                                  ▂
  ▆▁▁██▄▃▆██▅▆▆███▅▅███▅▇▆▆▅█▅▄▅▆▅▆▇▆▅▄▃▅▄▆▅▃▄▄▄▃▇▄▅▄▄▃▄▄▁▁▃▁▄ █
  715 ns        Histogram: log(frequency) by time       110 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Compare this to the benchmark results above.